In [1]:
# import numpy as np
# from lines.Line import Line
# from prototypes.models.PrototypeClassifierModels import CLASSIFIER_MODEL_2NN
# from training.CrossEntropyTraining import ClassifierTrainerWithCrossEntropyNoGen
# from datautils.LEOPARDEncoderUtils import get_labelled_LEOPARD_training_data as get_labelled_training_data
# from lines.LineGenerator import LineGenerator
# # from evaluation.Evaluator import Evaluator

# test_params = {
#   'type': 'cross_entropy',
#   'encoder': "bert",
# }

# trainer_params = {
#   'shuffle': True,
#   'num_workers': 0
# }

# batch_size = {
#   4: 4, 8: 8, 16: 12
# }

# epochs = {
#   4: {
#     2: 15, 3: 30, 4: 45, 5: 60, 6: 75
#   },
#   8: {
#     2: 30, 3: 40, 4: 50, 5: 70, 6: 100
#   },
#   16: {
#     2: 15, 3: 25, 4: 35, 5: 45, 6: 55
#   },
# }

# learning_rates = {
#   4: 5e-3,
#   8: 1e-2,
#   16: 5e-2
# }

# training_params = {
#   'encoder': "bert",
#   'epochs': epochs,
#   'warmupSteps': 100,
#   'reduction': 'none',
#   'learning_rate': learning_rates,
#   'printValidationPlot': True,
#   'printValidationLoss': False
# }

# for category in ["rating_dvd"]:
#     training_params['category'] = category
#     for episode in range(10):
#         training_params['episode'] = episode
#         for shot in [4]:
#             training_params['shot'] = shot
#             trainer_params['batch_size'] = batch_size[shot]
#             training_encodings, training_labels, label_keys = get_labelled_training_data(category, shot, episode)
            
#             if training_encodings.shape[0] == 0:
#                 continue
            
#             training_set = {}
#             training_set['encodings'] = training_encodings
#             training_set['labels'] = training_labels
            
#             lineGenerator = LineGenerator(training_set, CLASSIFIER_MODEL_2NN)
#             classifierTrainer = ClassifierTrainerWithCrossEntropyNoGen(lineGenerator, trainer_params, training_params, label_keys)
#             classifierTrainer.trainPrototypes(training_params, trainer_params, training_set)
#             lines = classifierTrainer.getLines()
            
# #             evaluator = Evaluator(label_keys, test_params)
# #             evaluator.evaluate(category, shot, episode)

In [2]:
from datasets import load_dataset
from training_datasets.GLUEDataset import GLUEDataset

cola = load_dataset('glue','cola')
print(cola)
# sst2 = load_dataset('glue','sst2')
# print(sst2)
mrpc = load_dataset('glue', 'mrpc')
print(mrpc)
pt_cola = GLUEDataset([cola, mrpc], 'train')
print(pt_cola)

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 1061.67it/s]


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})


Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 1488.93it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


In [3]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

# sampler = FewShotEpisodeSampler(pt_cola, kShot=2, nWay=4, shuffle=True)
# train_data_loader = data.DataLoader(
#     pt_cola,
#     batch_sampler=sampler,
#     num_workers=1,
# )

# data, targets = next(iter(train_data_loader))

# for i in range(5):
#     data, targets = next(iter(train_data_loader))
#     print(data)
#     print(targets)

train_protomaml_sampler = FewShotEpisodeBatchSampler(pt_cola, kShot=2, nWay=4, batchSize=3, shuffle=True)
train_protomaml_loader = data.DataLoader(
    pt_cola, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=1
)

for i in range(1):
    batch = next(iter(train_protomaml_loader))
    for episode_i in range(len(batch[0])):
        data, labels = batch[0][episode_i], batch[1][episode_i]
        supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 

In [4]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [5]:
from training.models.FOMAML import FOMAML

fomaml = FOMAML(None, None, None, None, None)
fomaml.training_step(next(iter(train_protomaml_loader)), 0)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predic

tensor([[ 0.8513,  0.2463, -0.0664,  ..., -0.4092,  0.5120, -0.4031],
        [ 0.7107,  0.1759, -0.5804,  ..., -0.5555,  0.1123, -0.1246],
        [ 0.6555, -0.0618,  0.4311,  ..., -0.3217,  0.4120,  0.0432],
        [ 0.4278,  0.0900, -0.4331,  ..., -0.4846,  0.2068,  0.0856]],
       grad_fn=<MulBackward0>)
tensor([[ 0.8513,  0.2463, -0.0664,  ..., -0.4092,  0.5120, -0.4031],
        [ 0.7107,  0.1759, -0.5804,  ..., -0.5555,  0.1123, -0.1246],
        [ 0.6555, -0.0618,  0.4311,  ..., -0.3217,  0.4120,  0.0432],
        [ 0.4278,  0.0900, -0.4331,  ..., -0.4846,  0.2068,  0.0856]])
tensor([-276.6008, -257.7414, -268.7680, -257.0078])
tensor([[ 0.8513,  0.2463, -0.0664,  ..., -0.4092,  0.5120, -0.4031],
        [ 0.7107,  0.1759, -0.5804,  ..., -0.5555,  0.1123, -0.1246],
        [ 0.6555, -0.0618,  0.4311,  ..., -0.3217,  0.4120,  0.0432],
        [ 0.4278,  0.0900, -0.4331,  ..., -0.4846,  0.2068,  0.0856]])
tensor([-276.6008, -257.7414, -268.7680, -257.0078])
tensor([[ 0.8513,  0

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[ 0.4641,  0.2270, -0.0442,  ..., -0.5645,  0.5754, -0.6869],
        [ 0.5371, -0.1211, -0.1727,  ..., -0.5288,  0.1132,  0.1254],
        [ 1.0453,  0.0924,  0.3596,  ..., -0.4572,  0.1936,  0.1107],
        [ 0.4543, -0.0549, -0.1443,  ..., -0.4803,  0.0073,  0.3440]],
       grad_fn=<MulBackward0>)
tensor([[ 0.4641,  0.2270, -0.0442,  ..., -0.5645,  0.5754, -0.6869],
        [ 0.5371, -0.1211, -0.1727,  ..., -0.5288,  0.1132,  0.1254],
        [ 1.0453,  0.0924,  0.3596,  ..., -0.4572,  0.1936,  0.1107],
        [ 0.4543, -0.0549, -0.1443,  ..., -0.4803,  0.0073,  0.3440]])
tensor([-262.8743, -264.0276, -256.0292, -270.3751])
tensor([[ 0.4641,  0.2270, -0.0442,  ..., -0.5645,  0.5754, -0.6869],
        [ 0.5371, -0.1211, -0.1727,  ..., -0.5288,  0.1132,  0.1254],
        [ 1.0453,  0.0924,  0.3596,  ..., -0.4572,  0.1936,  0.1107],
        [ 0.4543, -0.0549, -0.1443,  ..., -0.4803,  0.0073,  0.3440]])
tensor([-262.8743, -264.0276, -256.0292, -270.3751])
tensor([[ 0.4641,  0

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[ 0.8482, -0.0148,  0.2314,  ..., -0.3818,  0.5245, -0.0465],
        [ 0.6352, -0.4698, -0.1750,  ..., -0.4889,  0.2693,  0.1165],
        [ 0.8977, -0.0496, -0.2737,  ..., -0.1827,  0.3254,  0.2449],
        [ 0.1257, -0.2261, -0.3078,  ..., -0.1631,  0.0702, -0.0879]],
       grad_fn=<MulBackward0>)
tensor([[ 0.8482, -0.0148,  0.2314,  ..., -0.3818,  0.5245, -0.0465],
        [ 0.6352, -0.4698, -0.1750,  ..., -0.4889,  0.2693,  0.1165],
        [ 0.8977, -0.0496, -0.2737,  ..., -0.1827,  0.3254,  0.2449],
        [ 0.1257, -0.2261, -0.3078,  ..., -0.1631,  0.0702, -0.0879]])
tensor([-260.2888, -261.9232, -274.5767, -261.5992])
tensor([[ 0.8482, -0.0148,  0.2314,  ..., -0.3818,  0.5245, -0.0465],
        [ 0.6352, -0.4698, -0.1750,  ..., -0.4889,  0.2693,  0.1165],
        [ 0.8977, -0.0496, -0.2737,  ..., -0.1827,  0.3254,  0.2449],
        [ 0.1257, -0.2261, -0.3078,  ..., -0.1631,  0.0702, -0.0879]])
tensor([-260.2888, -261.9232, -274.5767, -261.5992])
tensor([[ 0.8482, -0

In [6]:
import torch

x = torch.Tensor([1,2,3,4,5,6,7,3,4,5,3])
labels = torch.Tensor([1,2,2,2,1,2,1,1,2,1,1])
print(x[torch.where(labels == 1)[0]])
print(x[torch.where(labels == 1)[0]].mean(dim=0))

tensor([1., 5., 7., 3., 5., 3.])
tensor(4.)
